# Time Series - GVB

## Imports

In [1]:
import pandas as pd

## Import File

In [2]:
dataset = pd.read_csv("../../../Data_thesis/Full_Datasets/TimeSeriesGVB.csv")

In [3]:
dataset.head()

,Arrivals,Departures,Station
0,189.810576,169.318074,Nieuwmarkt
1,485.702447,523.926598,Dam
2,82.026046,94.108129,Spui
3,100.131807,95.400158,Nieuwezijds Kolk
4,2401.857932,2654.147593,Centraal Station


## Functions

In [ ]:
def plotTimeSeries(df):
    
    #Variables
    x_column = 'SensorLongitude'
    y_column = 'SensorLatitude'
    bubble_column = 'Sensor'
    time_column = 'Hour'
    size_column = 'CrowdednessCount' 
    date = pd.Timestamp.strftime(dataset["Date"][0], format="%Y-%m-%d")

    #Initialize grid
    grid = pd.DataFrame()
    
    figure = bubbleplot(dataset=dataset, x_column=x_column, y_column=y_column, 
        bubble_column=bubble_column, size_column=size_column, time_column=time_column, color_column=bubble_column,
        x_title="Sensor Longitude", y_title="Sensor Latitude", title='Crowdedness Counts Amsterdam - ' + date,
        x_logscale=False, scale_bubble=3, height=650, x_range=[min(dataset[x_column])-0.001, max(dataset[x_column])+0.001],
                       y_range=[min(dataset[y_column])-0.001, max(dataset[y_column])+0.001])

    plotly.offline.iplot(figure)